# Writing data to and reading data from a Database using Python

## Libraries and settings

In [30]:
# Libraries
import os
import sqlite3
import fnmatch
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

# Ignore warnings
import warnings
warnings.filterwarnings("ignore")

# Function to close a sqlite db-connection
def check_conn(conn):
     try:
        conn.cursor()
        return True
     except Exception as ex:
        return False

# Get current working directory
print(os.getcwd())

/workspaces/data_analytics/Week_02


## Create sqlite data base

In [31]:
# Create data base
conn = sqlite3.connect('supermarkets_database.db') 
cursor = conn.cursor()

# Show dbs in the directory
flist = fnmatch.filter(os.listdir('.'), '*.db')
for i in flist:
    print(i)

supermarkets_database.db
apartment_database.db


## Create SQL-table in the database

In [32]:
cursor.execute('''CREATE TABLE IF NOT EXISTS supermarkets_table (Brand VARCHAR(200),
                                                                City VARCHAR(200),
                                                                Street VARCHAR(200),
                                                                Housenumber INT(10),
                                                                Postcode INT(14),
                                                                Opening_Hours VARCHAR(200))''')        
# Confirm changes to the table
conn.commit()

## Read raw data from file to data frame

In [33]:
df1 = pd.read_json('supermarkets.json', encoding='utf-8')
df1.head(5)

,type,id,lat,lon,tags
0,node,33126515,47.155616,9.037915,"{'brand': 'Spar', 'brand:wikidata': 'Q610492',..."
1,node,36726161,47.226191,8.980329,"{'addr:city': 'Uznach', 'addr:housenumber': '2..."
2,node,39768209,47.225069,8.969981,"{'addr:city': 'Uznach', 'addr:postcode': '8730..."
3,node,39947904,47.376732,8.542161,"{'addr:city': 'Zürich', 'addr:country': 'CH', ..."
4,node,48932835,47.375020,8.522895,"{'addr:city': 'Zürich', 'addr:housenumber': '7..."


## Convert data frame to appropriate format

In [34]:
# Save the tags as a data frame
df = pd.DataFrame.from_records(df1.tags)
df = df[['brand', 'addr:city', 'addr:street', 'addr:housenumber', 'addr:postcode', 'opening_hours']]

# Rename selected columns
df = df.rename(columns={'addr:city': 'city',
                          'addr:street':'street',
                          'addr:housenumber': 'housenumber',
                          'addr:postcode': 'postcode'})

# Filter all elements with missing values
df = df.loc[(pd.isna(df['city']) | pd.isna(df['street']) | pd.isna(df['housenumber']) |
             pd.isna(df['postcode']) | pd.isna(df['opening_hours'])) == False]

# Show first records of data frame
df.head()

,brand,city,street,housenumber,postcode,opening_hours
1,Migros,Uznach,Zürcherstrasse,25,8730,"Mo-Th 08:00-19:00, Fr 08:00-20:00, Sa 07:30-17..."
3,Coop,Zürich,Bahnhofbrücke,1,8001,Mo-Sa 06:00-22:00
4,Migros,Zürich,Wengistrasse,7,8004,Mo-Sa 08:00-21:00; PH off
7,Migros,Winterthur,Zürcherstrasse,102,8406,Mo-Fr 07:30-20:00; PH off; Sa 08:00-19:00
8,ALDI,Zürich,Albisstrasse,81,8038,Mo-Sa 07:30-21:00


## Write data to the SQL-table in data base

In [35]:
df.to_sql(name = 'supermarkets_table',
          con = conn,
          index = False,
          if_exists = 'replace')

1098

## Query the SQL-table

In [36]:
# Query the SQL-table
cursor.execute('''SELECT *
               FROM supermarkets_table
               WHERE city == 'Winterthur' ''')

df = pd.DataFrame(cursor.fetchall(), 
                  columns=['Brand','City','Street','Housenumber','Postcode','Opening_Hours'])    
df

,Brand,City,Street,Housenumber,Postcode,Opening_Hours
0,Migros,Winterthur,Zürcherstrasse,102,8406,Mo-Fr 07:30-20:00; PH off; Sa 08:00-19:00
1,Migros,Winterthur,Schaffhauserstrasse,152,8400,"Mo-Fr 08:30-20:00, Sa 08:00-18:00; PH off"
2,Migros,Winterthur,Wülflingerstrasse,71,8400,"Mo-Fr 07:30-20:00, Sa 08:00-18:00; PH off"
3,None,Winterthur,Bankstrasse,8/12,8400,"Mo-Fr 08:30-20:00; Sa 08:00-20:00; Su,PH off"
4,None,Winterthur,Steinberggasse,18,8400,"Tu,Fr 08:30-13:00,14:00-18:00; We,Th 09:00-13:..."
5,Migros,Winterthur,Hinterdorfstrasse,40,8405,"Mo-We 08:00-19:00; Th,Fr 08:00-20:00; Sa 08:00..."
6,Denner,Winterthur,Hinterdorfstrasse,40,8405,Mo-We 08:30-19:00; Th-Fr 08:30-20:00; Sa 08:00...
7,Migros,Winterthur,Strickerstrasse,3,8400,"Mo-Fr 08:00-20:00, Sa 08:00-18:00; PH off"
8,Migros,Winterthur,Stadthausstrasse,31,8400,"Mo-Fr 07:00-20:00, Sa 08:00-18:00; PH off"
9,ALDI,Winterthur,Archplatz,2,8400,Mo-Sa 08:00-20:00


## Close db connection (if open)

In [37]:
# Close db connection (if open)
try:
    if check_conn(conn):
        conn.close()
    else:
        pass
except:
    pass

# Status (True = open, False = closed)
print(check_conn(conn))

False


### Jupyter notebook --footer info-- (please always provide this at the end of each submitted notebook)

In [38]:
import os
import platform
import socket
from platform import python_version
from datetime import datetime

print('-----------------------------------')
print(os.name.upper())
print(platform.system(), '|', platform.release())
print('Datetime:', datetime.now().strftime("%Y-%m-%d %H:%M:%S"))
print('Python Version:', python_version())
print('-----------------------------------')

-----------------------------------
POSIX
Linux | 6.8.0-1014-azure
Datetime: 2024-09-26 16:55:03
Python Version: 3.11.10
-----------------------------------
